# AI Controller: PromptLib

PromptLib is a library for creating AST runner programs for execution by the AICI AST VM. In addition to wrapping AICI AST functionality, PromptLib also adds higher level functionality that it compiles down to AST runner's primitives.

In this tutorial notebook, we'll show how to go over the basic usage of PromptLib, including prompting and generation, constrained generation with regexes and context free grammars,  branches, variables, and control flow.


## Setup

Please see the AICI README to install and run an AICI-enabled LLM service.

In [1]:
import promptlib as pl
import json

aici = pl.AICI(base_url="http://127.0.0.1:8080/v1/", wasm_runner_path="/workspaces/aici/declctrl/target/opt.wasm")

upload module... 3271kB -> 10165kB id:f289fbde


## Basic Usage


We'll begin by demonstrating how to run a simple PromptLib text generation program.

In [2]:
# Create a new program and set its endpoint to our AICI node
program = pl.PromptProgram(aici)

# A PromptLib program is built up by appending fixed text, generation
# requests, and other commands
p = program + "Please answer the following simple question:\n"
p += "Q: What is the meaning of life?\n"
p = p.gen(max_tokens = 20, stop="\n")

# Once we've finished building the program, we can run it. 
result = program.run()

# Results are returned in a structured representation.
# result[0] is the full text conversation
print(json.dumps(result[0], indent=4))


Please answer the following simple question:
Q: What is the meaning of life?
A: The meaning of life is to find your purpose.
Q: What is the meaning of
[
    "Please answer the following simple question:\nQ: What is the meaning of life?\nA: The meaning of life is to find your purpose.\nQ: What is the meaning of"
]


In addition to the generated text, the result also includes ``compiled'' AST program as well as the detailed token-by-token generation logs.

In [3]:
# print the compiled AST program, returned in result[1]
print(json.dumps(result[1], indent=4))

{
    "model": "",
    "prompt": "",
    "max_tokens": 200,
    "n": 1,
    "temperature": 0,
    "stream": true,
    "aici_module": "f289fbde2f5ae02987e838f877215d764a04114d278ba013f59d4f68a1c78e26",
    "aici_arg": {
        "steps": [
            {
                "Fixed": {
                    "text": {
                        "String": {
                            "str": "Please answer the following simple question:\n"
                        }
                    },
                    "tag": "_1",
                    "label": "_1"
                }
            },
            {
                "Fixed": {
                    "text": {
                        "String": {
                            "str": "Q: What is the meaning of life?\n"
                        }
                    },
                    "tag": "_2",
                    "label": "_2"
                }
            },
            {
                "Gen": {
                    "max_tokens": 20,
                  

In [4]:
# result[2] is the set of raw logs capturing token by token execution of the program
print(json.dumps(result[2], indent=4))

[
    {
        "id": "cmpl-92cd312f5edf45e0803c2d44f45c39fa",
        "object": "text_completion",
        "created": 1702627507,
        "model": "codellama/CodeLlama-13b-Instruct-hf",
        "choices": [
            {
                "index": 0,
                "text": "Please answer the following simple question:\n",
                "logprobs": null,
                "finish_reason": null,
                "logs": "dfa: 144 bytes\n[0] Fixed(\"Please answer the following simple question:\\n\", tag:TagName(\"_1\"), label:_1) [Fixed()] tok:0/inf\n[1] Fixed(\"Q: What is the meaning of life?\\n\", tag:TagName(\"_2\"), label:_2) [Fixed()] tok:0/inf\n[2] Gen(max_tokens:20, , tag:TagName(\"_1\"), label:_1) [Gen()] tok:0/20\n[3] Stop [Stop] tok:0/inf\nprompt: [1]\ntokenize_bytes: \"Please answer the following simple question:\\n\" -> [12148, 1234, 278, 1494, 2560, 1139, 29901, 13]\nfinish: [Fixed()] tok:8/inf \"Please answer the following simple question:\\n\"\n",
                "storage": 

Instead of generating arbitrary text, we can also restrict the gneration in various ways.  The simplest is to force the language model to choose between two or more options.  The model will then choose the most likely of the options.

In [5]:
program = pl.PromptProgram(aici)

p = program + "Please answer the following simple question:\n"
p += "Q: What is the best television program ever made?\n"

# Force the LLM to choose between the listed options
p = p.choose(["MacGyver", "Star Trek", "She-Ra"])

result = program.run()

# Results are returned in a structured representation
print(result[0])

Please answer the following simple question:
Q: What is the best television program ever made?
She-Ra
['Please answer the following simple question:\nQ: What is the best television program ever made?\nShe-Ra']


We can save the selected choice to a variable.  We can also do this with any gen() command too.

In [6]:
program = pl.PromptNode(aici)

p = program + "Please answer the following simple question:\n"
p += "Q: What is the best television program ever made?\n"

# We'll add a second argument that will copy the chosen text into a variable for easy access
p = p.choose(["MacGyver", "Star Trek", "She-Ra"], set_var="best_tv_show")

result = p.run()

# Results are returned in a structured representation
print(result.variables["best_tv_show"])

TypeError: choose() got an unexpected keyword argument 'set_var'

### switch to a summarization task to extract book titles and then generate summary info of the best one.

Once we've captured a choice or other generated text in a variable, we can manipulate the value and also use it later in the program too.

In [ ]:
program = pl.PromptNode(aici)

p = program + "Please answer the following simple question:\n"
p += "Q: What is the best book ever written?\n")

# We'll add a second argument that will copy the chosen text into a variable for easy access
p = p.gen(max_tokens = 5, set_var="best_book")

p += "Q: Who wrote the book " + "****" + "?\n"
p = p.gen(max_tokens= 10, set_var="best_book_author")

result = p.run()

# Results are returned in a structured representation
print(result)

## Constrained Generation

PromptLib supports several ways to constrain text generation, using regular expressions and context free grammars.

In [ ]:
#fixed generation with regex constraints (and see more detailed regex notebook)

program = pl.PromptNode(aici)

p = program + "Please guess a number"
p = p.gen(max_tokens = 1, regex = r"\d+", set_var="number")

result = p.run()

print(result)


We can combine fixed text with choices and regexes to generate text that matches a specific pattern, like JSON.

In [ ]:
program = pl.PromptNode(aici)

note = """
The 3 astronauts who flew to the International Space Station (ISS) on SpaceX's Crew Dragon spacecraft
finally reached their destination today (May 31). The crew successfully docked to the space station at
10:16 a.m. EDT (1416 GMT) today, about 19 hours after blasting off from NASA's Kennedy Space Center in
Florida on Saturday (May 30). The docking went smoothly, with the Crew Dragon's soft capture docking
mechanism attaching to the space station's International Docking Adapter automatically at 10:16 a.m. EDT
(1416 GMT). The spacecraft then completed a hard capture by retracting its soft capture hooks to firmly
latch onto the docking adapter at 10:27 a.m. EDT (1427 GMT)."""

p = program + "Please summarize and extract key factors from the following text, and return the result as JSON:\n"
p += "<text>" + note + "</text>\n"
p += "JSON:"
p += "{title: '"
p = p.gen(max_tokens = 10, set_var="title") + "', who: '"
p = p.gen(max_tokens = 10, set_var="who") + "', when: '"
p = p.gen(max_tokens = 10, set_var="when") + "', where: '"
p = p.gen(max_tokens = 10, set_var="where") + "', what: '"
p = p.gen(max_tokens = 10, set_var="what") + "', excitement_level: '"
p = p.gen(max_tokens = 10, regex = r"\d+", set_var="excitement_level") + "', quality_level: '"
p = p.choose(["high", "medium", "low"], set_var="quality_level") + "'}\n"

result = p.run()

print(result)

Here's a simple constrained generation example using a context-free grammar.

In [ ]:
program = pl.PromptNode(aici)

# load a grammar file
c_grammar = open("aici_abi/grammars/c.y", "r").read()

p = program + "Please write a C program to calculate the nth digit of pi\n"
p = p.gen(max_tokens = 200, cfg=c_grammar, set_var="code")

result = p.run()

print(result)

For more examples of regular expressions and context free grammars see the ** add the URL for the respective notebooks **

## Branches and control flow

##### introduce begin/end blocks

Sometimes, we may want to generate text for programmatic use, without keeping the text in the prompt afterwards.  In this case, we can use a begin/end block and set its hidden parameter to True.

In [ ]:
program = pl.PromptNode(aici)

text = """
    The Contoso factory floor has 3 machines running in sequence. The first machine takes
    as input potatoes and peels them.  The 2nd machine takes the peeled potatoes and cuts
    them into thin slices.  The 3rd machine takes the slices and fries them.
    Yesterday, a new batch of potatoes came in and the factory workers noticed that the
    3rd machine was not working properly.  The fries were coming out soggy and the
    factory workers were worried that the machine was broken.  The factory manager
    called the repairman, who came in and inspected the machine.  The repairman
    determined that the machine was working properly, but that the fries were soggy
    because the potatoes were too wet."
"""

p = program + "Please analyze the following situation and diagnose the failure:" + text + "\n"
p = p.begin(hidden=True) + "List the important factors relevant to diagnosing the failure:\n"
p = p.gen(max_tokens = 50, set_var="factors") + "\n"
p += "Analyze the factors and determine the root cause of the failure:\n"
p = p.gen(max_tokens = 50, set_var="root_cause") + "\n"
p += "Briefly summarize the cause:\n"
p = p.gen(max_tokens = 50, set_var="summary") + "\n"
p = p.end()
p = p.fixed(var="summary")

result = p.run()

print(result)

In [ ]:
Forking and recombining

In [ ]:
Fork-If (**)